# Реализуем методы для наивного байеса

Сгенерируем выборку, в которой каждый признак имеет некоторое своё распределение, параметры которого отличаются для каждого класса. Затем реализуем несколько методов для класса, который уже частично написан ниже:
- метод predict
- метод \_find\_expon\_params и \_get\_expon\_density для экспоненциального распределения
- метод \_find\_norm\_params и \_get\_norm\_probability для биномиального распределения

Для имплементации \_find\_something\_params изучите документацию функций для работы с этими распределениями в scipy.stats и используйте предоставленные там методы.

In [77]:
import numpy as np
import scipy
import scipy.stats

Сформируем параметры генерации для трех датасетов

In [78]:
func_params_set0 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                   ]

func_params_set1 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                   ]

func_params_set2 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                    (scipy.stats.norm, [dict(loc=0, scale=1), dict(loc=1, scale=2)]),
                   ]

def generate_dataset_for_nb(func_params_set=[], size = 2500, random_seed=0):
    '''
    Генерирует выборку с заданными параметрами распределений P(x|y).
    Число классов задается длиной списка с параметрами.
    Возвращает X, y, список с названиями распределений
    '''
    np.random.seed(random_seed) 

    X = []
    names = []
    
    for func, params in func_params_set:
        names.append(func.name) 
        f = []
        for i, param in enumerate(params):
            f.append(func.rvs(size=size, **param)) #  *args is used to send a non-keyworded variable length argument list to the function. **kwargs allows you to pass keyworded variable length of arguments to a function. 
        f = np.concatenate(f).reshape(-1,1) # New shape as (-1, 2). row unknown, column 2. 
        X.append(f)

    X = np.concatenate(X, 1) # half distributed with first params, half with second
    
    y = np.array([0] * size + [1] * size) # half first class, half second

    #shuffle_inds = np.random.choice(range(len(X)), size=len(X), replace=False)
    #X = X[shuffle_inds]
    #y = y[shuffle_inds]

    return X, y, names 

X, y, distrubution_names = generate_dataset_for_nb(func_params_set2)
X.shape, y.shape, distrubution_names

((5000, 3), (5000,), ['bernoulli', 'expon', 'norm'])

In [79]:
np.concatenate((X,np.array([y]).T), 1)

array([[ 0.        ,  0.45693771, -0.72582032,  0.        ],
       [ 0.        ,  0.35328666,  0.56347552,  0.        ],
       [ 0.        ,  0.07106601, -0.43563209,  0.        ],
       ...,
       [ 0.        ,  0.4261782 ,  3.01704945,  1.        ],
       [ 0.        ,  0.00722247,  2.52513833,  1.        ],
       [ 1.        ,  0.50391801, -0.37623252,  1.        ]])

In [80]:
scipy.stats.bernoulli.pmf(X[:,0], np.mean(X[2500:,1]))
scipy.stats.expon.pdf(X[:,1], scale=np.mean(X[2500:,1]))
scipy.stats.norm.pdf(X[:,2], loc=np.mean(X[2500:,2]), scale=np.std(X[2500:,2])) 

array([0.14016722, 0.19534797, 0.15659212, ..., 0.11711708, 0.14633245,
       0.15977091])

In [191]:
from collections import defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin

class NaiveBayes(BaseEstimator, ClassifierMixin):
    '''
    Реализация наивного байеса, которая помимо X, y
    принимает на вход во время обучения 
    виды распределений значений признаков
    '''
    def __init__(self):  # For example, __file__ indicates the location of Python file, __eq__ is executed when a == b expression is excuted. 
        pass # class C: pass   -  a class with no methods (yet)
    
    # bernoulli
    
    def _find_bernoulli_params(self, x):           #x, *_, y = (1, 2, 3, 4, 5) # x = 1, y = 5 
        '''
        метод возвращает найденный параметр `p`
        распределения scipy.stats.bernoulli
        '''
        return dict(p=np.mean(x))
                                                     # single_trailing_underscore_: This convention could be used for avoiding conflict with Python keywords or built-ins. You might not use it often.
    def _get_bernoulli_probability(self, x, params): # _single_leading_underscore: This convention is used for declaring private variables, functions, methods and classes in a module. Anything with this convention are ignored in from module import *. 
        '''
        метод возвращает вероятность x для данных
        параметров распределния
        '''
        return scipy.stats.bernoulli.pmf(x, **params) # Probability mass function.

    # exp
    
    def _find_expon_params(self, x):
        '''
        метод возвращает ML - оценку параметра `scale` 
        распределения scipy.stats.expon
        '''
        return dict(scale=np.mean(x)) 
    
    def _get_expon_density(self, x, params):
        '''
        метод возвращает плотность распределения x для данных
        параметров распределния
        '''
        return scipy.stats.expon.pdf(x, **params) # Probability density function.

    
    #norm

    def _find_norm_params(self, x):
        '''
        метод возвращает ML - оценку параметра `loc`, `scale` 
        распределения scipy.stats.norm
        '''
        return dict(loc=np.mean(x), scale=np.std(x))    
    
    def _get_norm_density(self, x, params):
        '''
        метод возвращает плотность распределения x для данных
        параметров распределния
        '''
        return scipy.stats.norm.pdf(x, **params) # Probability density function.   

    # get get
    
    def _get_params(self, x, distribution):
        '''
        x - значения из распределения,
        distribution - название распределения в scipy.stats
        '''
        if distribution == 'bernoulli':
            return self._find_bernoulli_params(x)
        elif distribution == 'expon':
            return self._find_expon_params(x)
        elif distribution == 'norm':
            return self._find_norm_params(x)
        else:
            raise NotImplementedError('Unknown distribution')
            
    def _get_probability_or_density(self, x, distribution, params):
        '''
        x - значения,
        distribytion - название распределения в scipy.stats,
        params - параметры распределения
        '''
        if distribution == 'bernoulli':
            return self._get_bernoulli_probability(x, params)
        elif distribution == 'expon':
            return self._get_expon_density(x, params)
        elif distribution == 'norm':
            return self._get_norm_density(x, params)
        else:
            raise NotImplementedError('Unknown distribution')

    # fit predict        
            
    def fit(self, X, y, distrubution_names):
        '''
        X - обучающая выборка,
        y - целевая переменная,
        feature_distributions - список названий распределений, 
        по которым предположительно распределны значения P(x|y)
        ''' 
        assert X.shape[1] == len(distrubution_names)
        assert set(y) == {0, 1}
        self.n_classes = len(np.unique(y)) # 2 classes
        self.distrubution_names = distrubution_names 
        
        self.y_prior = [(y == j).mean() for j in range(self.n_classes)]   # [0.5, 0.5]
        
        self.distributions_params = defaultdict(dict) # defaultdict(dict, {})
        
        for i in range(X.shape[1]):
            distribution = self.distrubution_names[i]
            for j in range(self.n_classes):
                values = X[y == j, i]
                self.distributions_params[j][i] = \
                    self._get_params(values, distribution)
        
        return self.distributions_params
    
    def predict(self, X):
        '''
        X - тестовая выборка
        '''
        assert X.shape[1] == len(self.distrubution_names)
                
        preds = []
            
        for i in range(X.shape[0]):
            
            poster = np.log(np.array(self.y_prior))  # [0.5, 0.5]
             
            for k in range(len(self.distrubution_names)): 
                
                proba = []
                
                for j in range(self.n_classes): 
                
                    pr = self._get_probability_or_density([X[i][k]], self.distrubution_names[k], self.distributions_params[j][k])
                    
                    proba.extend(pr)
                    
                poster = np.array(poster) + np.log(np.array(proba)) # [0.3, 0.7]
            
            preds.append(np.argmax(poster))
        
        
        # нужно реализовать подсчет аргмаксной формулы, по которой 
        # наивный байес принимает решение о принадлежности объекта классу
        # и применить её для каждого объекта в X
        
        # примечание: обычно подсчет этой формулы реализуют через 
        # её логарифмирование, то есть, через сумму логарифмов вероятностей, 
        # поскольку перемножение достаточно малых вероятностей будет вести
        # к вычислительным неточностям
        
        return preds

Проверим результат на примере первого распределения

In [192]:
X, y, distrubution_names = generate_dataset_for_nb(func_params_set0)

nb = NaiveBayes()
nb.fit(X, y, ['bernoulli'])

# defaultdict(dict, {0: {0: {'p': 0.1128}}, 1: {0: {'p': 0.482}}})

defaultdict(dict, {0: {0: {'p': 0.1128}}, 1: {0: {'p': 0.482}}})

In [193]:
from sklearn.metrics import f1_score  # The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is :  F1 = 2 * (precision * recall) / (precision + recall),  where preision = true positives/positives; recall = true poisitives/relevant (relevant = true positives + false negatives) ) 

prediction = nb.predict(X)
score = f1_score(y, prediction)
print('{:.4f}'.format(score))

# 0.6045

0.6045


https://en.wikipedia.org/wiki/File:Precisionrecall.svg

In [194]:
X, y, distrubution_names = generate_dataset_for_nb(func_params_set2)

nb = NaiveBayes()
nb.fit(X, y, ['bernoulli', 'expon', 'norm'])

defaultdict(dict,
            {0: {0: {'p': 0.1128},
              1: {'scale': 0.9852643613643739},
              2: {'loc': -0.031953073806878646, 'scale': 0.9861590182151528}},
             1: {0: {'p': 0.482},
              1: {'scale': 0.2960380256457503},
              2: {'loc': 0.9528354019954841, 'scale': 2.004030301628345}}})

In [195]:
from sklearn.metrics import f1_score  # The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is :  F1 = 2 * (precision * recall) / (precision + recall),  where preision = true positives/positives; recall = true poisitives/relevant (relevant = true positives + false negatives) ) 

prediction = nb.predict(X)
score = f1_score(y, prediction)
print('{:.4f}'.format(score))


0.7890


# Ответы для формы

Ответом для формы должны служить числа, которые будут выведены ниже. Все ответы проверены: в этих примерах получается одинаковый результат и через сумму логарифмов, и через произведение вероятностей.

In [196]:
scipy.stats.bernoulli.name

for fps in (func_params_set0 * 2,
            func_params_set1, 
            func_params_set2):
    

    X, y, distrubution_names = generate_dataset_for_nb(fps)
    
    nb = NaiveBayes()
    nb.fit(X, y, distrubution_names)
    prediction = nb.predict(X)
    score = f1_score(y, prediction)
    print('{:.4f}'.format(score))

0.7615
0.7511
0.7890
